In [1]:
%load_ext autoreload
%load_ext autotime

time: 247 µs (started: 2024-10-08 14:01:22 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 983 µs (started: 2024-10-08 14:01:22 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/cm1_1_48_20240926/cm1_1_48_20240926-COMPACT/*.pfw.gz"

time: 738 µs (started: 2024-10-08 14:01:22 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-deepspeed-dlio-scr-node-16"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/deepspeed-dlio-scr-node-16"
trace_path = "/p/lustre3/iopp/dftracer-traces/v1.0.5-develop/corona/megatron_deepspeed-1-100-1_16_8_scr/megatron_deepspeed-1-100-1_16_8_scr-COMPACT/*.pfw.gz"


time: 619 µs (started: 2024-10-08 14:01:22 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 9.1 s (started: 2024-10-08 14:01:22 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    threshold=cfg.threshold,
    view_types=cfg.view_types,
)

time: 14min 12s (started: 2024-10-08 14:01:31 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          338.02 seconds                                                                                │
│  I/O Time         35.27 seconds                                                                                 │
│                   ├── Read - 4.45 seconds (12.60%)                                                              │
│                   ├── Write - 0.04 seconds (0.10%)                                                              │
│                   └── Metadata - 30.96 seconds (87.77%)                                                         │
│  I/O Operations   25,820,495 ops                                                                                │
│                   ├── Read - 1,316,696 ops (5.10%)                                                              │
│                   ├── Write - 544,799 ops (2.11%)                                                               │
│                   └── Metadata - 23,462,488 ops (90.87%)                                                        │
│  I/O Size         8.15 GiB                                                                                      │
│                   ├── Read - 7.58 GiB (93.00%)                                                                  │
│                   └── Write - 584.08 MiB (7.00%)                                                                │
│  Read Requests    4 kiB-1 MiB - 1,316,170 ops                                                                   │
│                   ├── <4 kiB - 662,135 ops (50.31%)                                                             │
│                   ├── 4-16 kiB - 371,411 ops (28.22%)                                                           │
│                   ├── 16-64 kiB - 244,448 ops (18.57%)                                                          │
│                   ├── 64-256 kiB - 35,872 ops (2.73%)                                                           │
│                   └── 256 kiB-1 MiB - 2,304 ops (0.18%)                                                         │
│  Write Requests   4 kiB-64 kiB - 544,799 ops                                                                    │
│                   ├── <4 kiB - 535,197 ops (98.24%)                                                             │
│                   ├── 4-16 kiB - 130 ops (0.02%)                                                                │
│                   └── 16-64 kiB - 9,472 ops (1.74%)                                                             │
│  Nodes            16 nodes                                                                                      │
│                   ├── 39194 - 35.27 s (100.00%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,491 ops         │
│                   │   (6.25%)                                                                                   │
│                   ├── 12967 - 34.65 s (98.23%) - 485.23 MiB/37.31 MiB R/W (5.81/0.45%) - 1,617,918 ops (6.27%)  │
│                   ├── 35656 - 34.40 s (97.51%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,505 ops (6.25%)  │
│                   ├── 60580 - 33.30 s (94.39%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,514 ops (6.25%)  │
│                   ├── 51714 - 33.12 s (93.89%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,513 ops (6.25%)  │
│                   ├── 11383 - 32.50 s (92.13%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,503 ops (6.25%)  │
│                   ├── 23424 - 32.50 s (92.12%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,506 ops (6.25%)  │
│                   ├── 62039 - 32.47 s (92.05%) - 485.24 MiB/36.45 MiB R/W (5.81/0.44%) - 1,613,514 ops (6.25%)  │
│                   ├── 33196 - 32.38 s (91.79%) - 485.2

time: 9min 56s (started: 2024-10-08 14:15:43 -07:00)
